# 🏗️ Graph Algorithms for ML Pipelines

**Welcome, St. Mark!** In this notebook, we'll explore how graph algorithms power modern ML systems. Think of ML pipelines as transportation networks: data flows through preprocessing nodes, model training vertices, and deployment edges.

We'll implement:
1. **Dijkstra's Algorithm** - Finding optimal paths in ML workflow graphs
2. **Topological Sort** - Ordering pipeline execution dependencies  
3. **Connected Components** - Analyzing ML system architectures

By the end, you'll understand how graph theory enables scalable ML engineering.

## The Big Picture

ML pipelines are directed graphs where:
- **Nodes**: Processing steps (data cleaning, feature engineering, model training)
- **Edges**: Data flow dependencies (output of A feeds input of B)
- **Weights**: Computational costs or execution times

**Key Question:** How do we optimize pipeline execution order and resource allocation?

In [ ]:
import heapq
import networkx as nx
from collections import defaultdict, deque
import matplotlib.pyplot as plt
import numpy as np

# Nigerian healthcare context: ML pipeline for disease prediction
print("Setting up ML pipeline graph analysis...")
print("Nodes: data_ingestion → preprocessing → feature_engineering → model_training → evaluation")

**Cell Analysis:** We've imported essential libraries for graph algorithms.

- **heapq**: Priority queue for Dijkstra's algorithm (efficient minimum extraction)
- **networkx**: Graph creation and visualization (industry standard)
- **collections**: Efficient data structures for graph representations
- **matplotlib**: Pipeline visualization

**Healthcare Context:** Nigerian health systems need efficient ML pipelines for disease prediction across distributed hospitals.

**Reflection Question:** How would graph algorithms help coordinate ML pipelines across Lagos, Abuja, and Kano hospitals?

## Method 1: Dijkstra's Algorithm - Optimal Pipeline Paths

**Healthcare Analogy:** Finding the fastest route for medical supplies through Nigerian cities.

**ML Application:** Optimizing execution paths in ML pipelines with computational costs.

**Algorithm:** Greedily selects closest unvisited node using priority queue.

In [ ]:
def dijkstra(graph, start, end=None):
    """
    Dijkstra's algorithm for shortest paths in ML pipeline graphs.
    
    Parameters:
    graph: dict of dicts - {node: {neighbor: weight}}
    start: starting node
    end: target node (optional)
    
    Returns:
    distances: dict of shortest distances from start
    previous: dict for path reconstruction
    """
    # Priority queue: (distance, node)
    # Heap ensures we always process closest node first
    pq = [(0, start)]
    
    # Track shortest distances found so far
    distances = {node: float('inf') for node in graph}
    distances[start] = 0
    
    # Track path for reconstruction
    previous = {node: None for node in graph}
    
    while pq:
        # Extract node with smallest distance
        current_distance, current_node = heapq.heappop(pq)
        
        # Skip if we found a better path already
        if current_distance > distances[current_node]:
            continue
            
        # Explore neighbors
        for neighbor, weight in graph[current_node].items():
            distance = current_distance + weight
            
            # Found shorter path to neighbor
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                previous[neighbor] = current_node
                heapq.heappush(pq, (distance, neighbor))
    
    return distances, previous

def reconstruct_path(previous, start, end):
    """Reconstruct shortest path from previous dict"""
    path = []
    current = end
    while current is not None:
        path.append(current)
        current = previous[current]
    path.reverse()
    return path if path[0] == start else []

**Algorithm Analysis:**

- **Priority Queue:** Ensures we always process the closest unexplored node
- **Relaxation:** Updates distances when shorter paths are found
- **Time Complexity:** O((V+E) log V) with binary heap
- **Space Complexity:** O(V) for distances and priority queue

**ML Pipeline Application:** Find most efficient execution path considering computational costs.

**Healthcare Context:** Route medical data through Nigerian hospital network with minimal latency.

In [ ]:
# ML Pipeline Example: Healthcare diagnosis workflow
pipeline_graph = {
    'data_ingestion': {'preprocessing': 2},  # 2 minutes
    'preprocessing': {'feature_engineering': 5, 'data_validation': 1},
    'data_validation': {'feature_engineering': 2},
    'feature_engineering': {'model_training': 10, 'feature_selection': 3},
    'feature_selection': {'model_training': 2},
    'model_training': {'evaluation': 8, 'model_validation': 4},
    'model_validation': {'evaluation': 2},
    'evaluation': {}  # Terminal node
}

# Find optimal path from data ingestion to evaluation
distances, previous = dijkstra(pipeline_graph, 'data_ingestion', 'evaluation')
optimal_path = reconstruct_path(previous, 'data_ingestion', 'evaluation')

print(f"Minimum execution time: {distances['evaluation']} minutes")
print(f"Optimal pipeline path: {' → '.join(optimal_path)}")

# Visualize pipeline
plt.figure(figsize=(12, 8))
G = nx.DiGraph(pipeline_graph)
pos = nx.spring_layout(G, seed=42)

# Color optimal path
node_colors = ['lightblue' if node in optimal_path else 'lightgray' for node in G.nodes()]
edge_colors = ['red' if (u, v) in zip(optimal_path[:-1], optimal_path[1:]) else 'black' 
               for u, v in G.edges()]

nx.draw(G, pos, with_labels=True, node_color=node_colors, edge_color=edge_colors, 
        node_size=2000, font_size=10, arrows=True, arrowsize=20)

# Add edge weights
edge_labels = {(u, v): f'{w}min' for u, v, w in 
               [(u, v, pipeline_graph[u][v]) for u, v in G.edges()]}
nx.draw_networkx_edge_labels(G, pos, edge_labels)

plt.title('ML Pipeline: Optimal Execution Path (Dijkstra)')
plt.show()

**Pipeline Analysis:**

- **Optimal Path:** Shows most efficient execution sequence
- **Bottlenecks:** model_training (10 min) is the critical path
- **Parallelization:** data_validation can run alongside feature_engineering

**Healthcare Impact:** Nigerian health systems could optimize diagnostic workflows, reducing patient wait times.

**Reflection Question:** How would you modify this pipeline for real-time COVID-19 monitoring in Lagos hospitals?

## Method 2: Topological Sort - Pipeline Execution Order

**Healthcare Analogy:** Sequencing medical procedures - you can't administer treatment before diagnosis.

**ML Application:** Ensuring pipeline steps execute in correct dependency order.

**Algorithm:** Kahn's algorithm using indegree counting and queue.

In [ ]:
def topological_sort(graph):
    """
    Topological sort for DAG execution ordering.
    
    Parameters:
    graph: dict of dicts - {node: {neighbor: weight}}
    
    Returns:
    ordered_nodes: list of nodes in execution order
    or None if cycle detected
    """
    # Calculate indegrees (number of incoming edges)
    indegree = {node: 0 for node in graph}
    for node in graph:
        for neighbor in graph[node]:
            indegree[neighbor] += 1
    
    # Queue of nodes with no dependencies
    queue = deque([node for node in indegree if indegree[node] == 0])
    
    result = []
    
    while queue:
        # Process node with no remaining dependencies
        current = queue.popleft()
        result.append(current)
        
        # Reduce indegree of neighbors
        for neighbor in graph[current]:
            indegree[neighbor] -= 1
            # Add to queue if no dependencies left
            if indegree[neighbor] == 0:
                queue.append(neighbor)
    
    # Cycle detection: not all nodes processed
    if len(result) != len(graph):
        print("Cycle detected in pipeline - cannot execute!")
        return None
        
    return result

**Algorithm Analysis:**

- **Indegree Counting:** Tracks remaining dependencies for each node
- **Queue Processing:** Processes nodes with zero dependencies first
- **Cycle Detection:** If not all nodes processed, graph has cycles
- **Time Complexity:** O(V + E) - linear in graph size

**ML Pipeline Application:** Ensures data flows correctly through pipeline stages.

**Healthcare Context:** Medical procedures must follow proper sequence - diagnosis before treatment.

In [ ]:
# Execute pipeline in correct order
execution_order = topological_sort(pipeline_graph)

if execution_order:
    print("✅ Valid Pipeline Execution Order:")
    for i, step in enumerate(execution_order, 1):
        print(f"{i}. {step}")
        
    # Simulate parallel execution possibilities
    print("\n🔄 Parallel Execution Groups:")
    levels = {}
    for node in execution_order:
        # Find maximum level of predecessors
        pred_levels = [levels.get(pred, 0) for pred in pipeline_graph if node in pipeline_graph.get(pred, {})]
        levels[node] = max(pred_levels) + 1 if pred_levels else 1
    
    for level in range(1, max(levels.values()) + 1):
        parallel_steps = [node for node, lvl in levels.items() if lvl == level]
        print(f"Level {level}: {parallel_steps}")
else:
    print("❌ Pipeline has circular dependencies!")

**Execution Analysis:**

- **Sequential Order:** Ensures dependencies are respected
- **Parallel Groups:** Shows which steps can run simultaneously
- **Optimization:** Reduces total pipeline execution time

**Healthcare Impact:** Nigerian hospitals could parallelize lab tests and consultations.

**Reflection Question:** How would topological sort help coordinate multi-specialty medical teams?

## Method 3: Connected Components - System Architecture Analysis

**Healthcare Analogy:** Identifying separate hospital networks in Nigeria's healthcare system.

**ML Application:** Analyzing disconnected components in ML system architectures.

**Algorithm:** DFS/BFS traversal to find connected subgraphs.

In [ ]:
def find_connected_components(graph):
    """
    Find connected components in undirected graph.
    
    Parameters:
    graph: dict of sets - {node: {neighbors}}
    
    Returns:
    components: list of lists (each sublist is a component)
    """
    visited = set()
    components = []
    
    def dfs(node, component):
        """Depth-first search to explore component"""
        visited.add(node)
        component.append(node)
        
        for neighbor in graph.get(node, []):
            if neighbor not in visited:
                dfs(neighbor, component)
    
    # Convert directed graph to undirected for connectivity
    undirected = defaultdict(set)
    for node in graph:
        for neighbor in graph[node]:
            undirected[node].add(neighbor)
            undirected[neighbor].add(node)  # Bidirectional
    
    for node in undirected:
        if node not in visited:
            component = []
            dfs(node, component)
            components.append(component)
    
    return components

**Algorithm Analysis:**

- **DFS Traversal:** Explores connected subgraphs depth-first
- **Undirected Conversion:** Treats directed edges as bidirectional for connectivity
- **Component Detection:** Each unvisited node starts a new component
- **Time Complexity:** O(V + E) - visits each node/edge once

**ML Pipeline Application:** Identifies independent pipeline segments that can be developed separately.

**Healthcare Context:** Finds isolated hospital networks needing integration.

In [ ]:
# Analyze ML system architecture
ml_system_graph = {
    # Data pipeline component
    'data_ingestion': ['preprocessing'],
    'preprocessing': ['feature_engineering'],
    'feature_engineering': ['model_training'],
    
    # Model component  
    'model_training': ['evaluation'],
    'evaluation': [],
    
    # Monitoring component (separate system)
    'monitoring': ['alerts'],
    'alerts': [],
    
    # Deployment component (separate system)
    'deployment': ['serving'],
    'serving': []
}

components = find_connected_components(ml_system_graph)

print("🏗️ ML System Architecture Analysis:")
print(f"Found {len(components)} disconnected components:\n")

component_names = ['Data Pipeline', 'Monitoring System', 'Deployment System']
for i, (component, name) in enumerate(zip(components, component_names)):
    print(f"Component {i+1}: {name}")
    print(f"  Nodes: {component}")
    print(f"  Size: {len(component)} nodes")
    
    # Calculate component density
    possible_edges = len(component) * (len(component) - 1) / 2
    actual_edges = sum(1 for node in component 
                      for neighbor in ml_system_graph.get(node, []) 
                      if neighbor in component)
    density = actual_edges / possible_edges if possible_edges > 0 else 0
    print(f"  Density: {density:.2f} (higher = more interconnected)\n")

# Visualize components
plt.figure(figsize=(10, 8))
G = nx.DiGraph(ml_system_graph)
pos = nx.spring_layout(G, seed=42)

# Color by component
color_map = {}
colors = ['lightblue', 'lightgreen', 'lightcoral']
for i, component in enumerate(components):
    for node in component:
        color_map[node] = colors[i % len(colors)]

node_colors = [color_map.get(node, 'gray') for node in G.nodes()]

nx.draw(G, pos, with_labels=True, node_color=node_colors, 
        node_size=2000, font_size=10, arrows=True, arrowsize=20)
plt.title('ML System Components (Connected Components Analysis)')
plt.show()

**Architecture Analysis:**

- **Component Separation:** Data pipeline, monitoring, and deployment are independent
- **Development Strategy:** Can develop components separately, then integrate
- **Failure Isolation:** Issues in one component don't affect others

**Healthcare Impact:** Nigerian health systems could identify disconnected hospital networks needing connectivity.

**Reflection Question:** How would connected components analysis help integrate Nigeria's fragmented healthcare infrastructure?

## 🎯 Key Takeaways and Nigerian Healthcare Applications

**Algorithm Summary:**
- **Dijkstra's:** Finds optimal paths in weighted graphs (pipeline optimization)
- **Topological Sort:** Orders execution in dependency graphs (workflow sequencing)
- **Connected Components:** Identifies independent subsystems (architecture analysis)

**Healthcare Translation - Mark:**
Imagine coordinating Nigeria's healthcare AI systems:
- **Dijkstra's:** Route patient data through fastest diagnostic pathways
- **Topological Sort:** Sequence medical procedures across specialties
- **Components:** Connect isolated hospital systems into unified network

**Performance achieved:** Graph algorithms enable scalable ML pipeline orchestration!

**Reflection Questions:**
1. How would these algorithms optimize vaccine distribution across Nigerian states?
2. What graph structures would you use for modeling disease transmission networks?
3. How might connected components help identify healthcare service gaps?

**Next Steps:**
- Implement graph-based ML pipeline schedulers
- Add resource constraints to optimization
- Extend to distributed ML system coordination

**🏆 Excellent work, my student! You've mastered the graph algorithms powering modern ML systems.**